# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/germee/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/germee/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/germee/Desktop/courses/aie7-/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/germee/Desktop/courses/aie7-/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/germee/Desktop/courses/aie7-/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '6f651d'. Skipping!
Property 'summary' already exists in node '67119a'. Skipping!
Property 'summary' already exists in node '704be3'. Skipping!
Property 'summary' already exists in node '187a72'. Skipping!
Property 'summary' already exists in node '969b49'. Skipping!
Property 'summary' already exists in node '6bb5b4'. Skipping!
Property 'summary' already exists in node 'da0535'. Skipping!
Property 'summary' already exists in node 'a84f1c'. Skipping!
Property 'summary' already exists in node '2f66c4'. Skipping!
Property 'summary' already exists in node 'cc74b1'. Skipping!
Property 'summary' already exists in node 'bcc1f5'. Skipping!
Property 'summary' already exists in node 'f41ea6'. Skipping!
Property 'summary' already exists in node '1af0e4'. Skipping!
Property 'summary' already exists in node '73661a'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '6f651d'. Skipping!
Property 'summary_embedding' already exists in node '704be3'. Skipping!
Property 'summary_embedding' already exists in node 'da0535'. Skipping!
Property 'summary_embedding' already exists in node '67119a'. Skipping!
Property 'summary_embedding' already exists in node '969b49'. Skipping!
Property 'summary_embedding' already exists in node '187a72'. Skipping!
Property 'summary_embedding' already exists in node '6bb5b4'. Skipping!
Property 'summary_embedding' already exists in node 'bcc1f5'. Skipping!
Property 'summary_embedding' already exists in node '2f66c4'. Skipping!
Property 'summary_embedding' already exists in node '73661a'. Skipping!
Property 'summary_embedding' already exists in node '1af0e4'. Skipping!
Property 'summary_embedding' already exists in node 'a84f1c'. Skipping!
Property 'summary_embedding' already exists in node 'f41ea6'. Skipping!
Property 'summary_embedding' already exists in node 'cc74b1'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [14]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [15]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What information is provided in Volume 2 regar...,"[Chapter 1 Academic Years, Academic Calendars,...","Volume 2 discusses academic year requirements,...",single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(a) about?,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,What is Volume 8 about in relation to clinical...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of Clinical Work in a Standard Term ...,single_hop_specifc_query_synthesizer
3,Is the FWS program disbursed on a payment peri...,[Non-Term Characteristics A program that measu...,"No, the payment period is not applicable to th...",single_hop_specifc_query_synthesizer
4,Volume 7 what is it?,[both the credit or clock hours and the weeks ...,The context does not provide specific informat...,single_hop_specifc_query_synthesizer
5,How do the regulatory citations support the ac...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...","The regulatory citations, specifically 34 CFR ...",multi_hop_abstract_query_synthesizer
6,"In the context of Title IV programs, how do no...",[<1-hop>\n\nInclusion of Clinical Work in a St...,"In Title IV programs, clinical work conducted ...",multi_hop_abstract_query_synthesizer
7,How do courses that do not begin and end withi...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Courses that do not begin and end within a ter...,multi_hop_abstract_query_synthesizer
8,Wha volume 2 and 7 are impordant for disbursme...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that Volume 2 covers acad...,multi_hop_specific_query_synthesizer
9,Chapter 1 and Chapter 3 include info about cli...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Chapter 1 explains that clinical work outside ...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '7f2160'. Skipping!
Property 'summary' already exists in node 'c939d8'. Skipping!
Property 'summary' already exists in node '5bb8e7'. Skipping!
Property 'summary' already exists in node 'af2b4b'. Skipping!
Property 'summary' already exists in node 'ca0a7d'. Skipping!
Property 'summary' already exists in node '3d3093'. Skipping!
Property 'summary' already exists in node 'b4b335'. Skipping!
Property 'summary' already exists in node 'deb4d6'. Skipping!
Property 'summary' already exists in node '5d0fc8'. Skipping!
Property 'summary' already exists in node '234bbe'. Skipping!
Property 'summary' already exists in node 'b3afd5'. Skipping!
Property 'summary' already exists in node '16fd80'. Skipping!
Property 'summary' already exists in node '03c740'. Skipping!
Property 'summary' already exists in node '78717b'. Skipping!


...

Property 'summary_embedding' already exists in node '7f2160'. Skipping!
Property 'summary_embedding' already exists in node 'ca0a7d'. Skipping!
Property 'summary_embedding' already exists in node '5d0fc8'. Skipping!
Property 'summary_embedding' already exists in node '3d3093'. Skipping!
Property 'summary_embedding' already exists in node 'af2b4b'. Skipping!
Property 'summary_embedding' already exists in node '5bb8e7'. Skipping!
Property 'summary_embedding' already exists in node 'c939d8'. Skipping!
Property 'summary_embedding' already exists in node '234bbe'. Skipping!
Property 'summary_embedding' already exists in node 'b3afd5'. Skipping!
Property 'summary_embedding' already exists in node '16fd80'. Skipping!
Property 'summary_embedding' already exists in node '03c740'. Skipping!
Property 'summary_embedding' already exists in node 'b4b335'. Skipping!
Property 'summary_embedding' already exists in node 'deb4d6'. Skipping!
Property 'summary_embedding' already exists in node '78717b'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [17]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the Knowledge Center in relation to ac...,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not provide a specific defini...,single_hop_specifc_query_synthesizer
1,What is the regulation 34 CFR 668.3(b) about i...,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(b) pertains to weeks of instructi...,single_hop_specifc_query_synthesizer
2,What is the significance of Volume 8 in the co...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,Is the FWS program a payment period based prog...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,Considering the requirements for defining acad...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The regulatory citations specify that for cred...,multi_hop_abstract_query_synthesizer
5,How does completing additional hours or weeks ...,[<1-hop>\n\nboth the credit or clock hours and...,Completing additional hours or weeks of instru...,multi_hop_abstract_query_synthesizer
6,how program length and scheduled payment perio...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that the scheduled paymen...,multi_hop_abstract_query_synthesizer
7,How do the program length and scheduled paymen...,[<1-hop>\n\nboth the credit or clock hours and...,Disbursement timing for federal student aid su...,multi_hop_abstract_query_synthesizer
8,How do Volume 2 and Volume 8 relate to the inc...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Volume 2 discusses the requirements for academ...,multi_hop_specific_query_synthesizer
9,How do Volume 2 and Volume 7 relate to the req...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Volume 2 discusses the requirements for establ...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [18]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [19]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [20]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [22]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [23]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [24]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [25]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [27]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [28]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context are:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (including student Federal PLUS Loans and parent PLUS Loans)  \n- Subsidized and Unsubsidized Federal Stafford Loans (made under the Federal Family Education Loan (FFEL) Program before July 1, 2010)  \n- Federal SLS Loans  \n- Federal PLUS Loans (also under FFEL Program before it ended)\n\nAdditionally, Direct Subsidized Loans are available only to undergraduate students, while graduate or professional students are eligible only for Direct Unsubsidized Loans and Direct PLUS Loans.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [29]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [30]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

## LangSmith Evaluation

In [31]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'weary-charge-75' at:
https://smith.langchain.com/o/7f931194-9c58-4a68-837e-86cd87ca9e76/datasets/08ebad5e-e5ca-40b6-a252-a428ab8ba971/compare?selectedSessions=5c0c976e-0c6e-475b-a1c4-86039e32875f




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Appendix A and Appendix B guide the dis...,Based on the provided context:\n\n- Appendix A...,None,Appendix B provides detailed guidance on disbu...,1,1,0,7.072362,c8aad3a3-a7fa-454a-8bbd-b8adc05a9f7b,4e23ba1f-ef0e-496b-81a1-e0abe34300f8
1,Volume 8 include clinical work in standard ter...,"Based on the provided context from Volume 8, c...",None,Volume 8 discusses the inclusion of clinical w...,1,1,0,6.334647,9d3e6433-1fb6-41f3-ab91-54c044217b69,461c2191-918b-4abd-b5fe-e4f8f4e5a799
2,How do Volume 2 and Volume 7 relate to the req...,"Volume 2, Chapter 2 provides information about...",None,Volume 2 discusses the requirements for establ...,1,0,0,2.921850,dfbd1141-82c3-427f-8d1f-f48dc97da8e8,e8a9a4dc-33e8-48a9-9239-fe2ebb7b5d92
3,How do Volume 2 and Volume 8 relate to the inc...,"Based on the provided context, Volume 8 discus...",None,Volume 2 discusses the requirements for academ...,0,0,0,6.360965,df22b6da-4d2f-4d4f-ab2b-9bd865584853,d65ed180-284e-4c29-b7d8-0a7f3df438cd
4,How do the program length and scheduled paymen...,"Based on the provided context, the timing of d...",None,Disbursement timing for federal student aid su...,1,1,0,10.292846,2d8f1362-eaf7-4250-8a85-ba8caa3e48ec,258f929f-2930-4798-820c-5733c3dd54d8
5,how program length and scheduled payment perio...,Based on the provided context:\n\nMultiple dis...,None,The context explains that the scheduled paymen...,1,1,0,6.872945,531d48f8-9b03-4d09-a9d1-f0314ac4c7ce,9d08dc69-716e-4039-968a-e47eb2af4259
6,How does completing additional hours or weeks ...,"Based on the provided context, completing addi...",None,Completing additional hours or weeks of instru...,1,1,0,7.768252,28fa19e3-f497-4f68-8bb2-8058634543d7,22b89123-e070-4304-a22b-0c6dda2a43a3
7,Considering the requirements for defining acad...,"Based on the provided context, the minimum wee...",None,The regulatory citations specify that for cred...,1,1,0,10.029360,8d0794ec-128e-4630-93ad-2d64a710663c,73cf358a-703e-43dd-811d-5103b52a6359
8,Is the FWS program a payment period based prog...,The FWS (Federal Work-Study) Program is not a ...,None,The payment period is applicable to all Title ...,1,1,0,2.984529,c84d7464-b8df-493d-8810-816ce03a02b2,1312a26f-fcc7-42fb-9364-11e8d7c66261
9,What is the significance of Volume 8 in the co...,Volume 8 provides additional guidance and disc...,None,Inclusion of clinical work in a standard term ...,1,1,0,4.312717,cf278c0c-f131-4c98-9d5e-27729ab87d7c,7ec1d7a3-7dc4-4336-88b3-1280a53e0d05


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [32]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [33]:
rag_documents = docs

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [36]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [37]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [38]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [39]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question. Based on the information you've shared, there are several kinds of loans available to students and their families to help cover educational costs:\n\n1. **Direct Subsidized Loans** – These are loans where the government pays the interest while you're in school at least half-time. The amount you can borrow is based on your financial need.\n\n2. **Direct Unsubsidized Loans** – These loans are available to students regardless of financial need, and interest starts accruing as soon as the loan is disbursed.\n\n3. **Direct PLUS Loans** – These are loans that parents of dependent students can take out to help cover the student's cost of attendance. They require the parent to meet certain eligibility requirements, and the loan amount cannot exceed the student's cost of attendance minus other financial aid.\n\nAdditionally, if a parent is unable to obtain a Direct PLUS Loan, dependent students may qualify for additional Direct Unsubsidized Loan amounts under the i

Finally, we can evaluate the new chain on the same test set!

In [40]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'memorable-star-45' at:
https://smith.langchain.com/o/7f931194-9c58-4a68-837e-86cd87ca9e76/datasets/08ebad5e-e5ca-40b6-a252-a428ab8ba971/compare?selectedSessions=22f7e02a-9fe0-4592-abaa-2c85b82973da




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Appendix A and Appendix B guide the dis...,Thank you for your thoughtful question. I unde...,None,Appendix B provides detailed guidance on disbu...,0,0,1,7.542619,c8aad3a3-a7fa-454a-8bbd-b8adc05a9f7b,8588524a-0a01-4179-98aa-76ccdb25be83
1,Volume 8 include clinical work in standard ter...,Thank you for your thoughtful question. Based ...,None,Volume 8 discusses the inclusion of clinical w...,1,1,1,4.165889,9d3e6433-1fb6-41f3-ab91-54c044217b69,a497657e-9ac8-4fe6-a297-267b2c27c7f4
2,How do Volume 2 and Volume 7 relate to the req...,Thank you for your thoughtful question. From t...,None,Volume 2 discusses the requirements for establ...,0,0,1,5.591963,dfbd1141-82c3-427f-8d1f-f48dc97da8e8,25cc0ba2-d2a1-48a5-8e52-bb88d4dae065
3,How do Volume 2 and Volume 8 relate to the inc...,Thank you for your thoughtful question—it's cl...,None,Volume 2 discusses the requirements for academ...,0,0,1,4.039655,df22b6da-4d2f-4d4f-ab2b-9bd865584853,ed72ca8c-b164-4ab2-955f-347868b7ebdc
4,How do the program length and scheduled paymen...,Thank you for your thoughtful question — I und...,None,Disbursement timing for federal student aid su...,1,1,1,10.507674,2d8f1362-eaf7-4250-8a85-ba8caa3e48ec,6e9d1753-1ade-453e-9aaf-47863c55d22b
5,how program length and scheduled payment perio...,Thank you for your thoughtful question. From t...,None,The context explains that the scheduled paymen...,1,0,1,8.492310,531d48f8-9b03-4d09-a9d1-f0314ac4c7ce,0705541a-6ed9-4e9f-8ed7-47a068b719b5
6,How does completing additional hours or weeks ...,Thank you for your thoughtful question. Based ...,None,Completing additional hours or weeks of instru...,1,1,1,10.672771,28fa19e3-f497-4f68-8bb2-8058634543d7,45e41f93-d37b-43c2-beaf-d019b27493de
7,Considering the requirements for defining acad...,Thank you for your thoughtful question. I can ...,None,The regulatory citations specify that for cred...,1,0,1,7.108033,8d0794ec-128e-4630-93ad-2d64a710663c,27d2563f-2ea9-494e-99e6-c57223a731cd
8,Is the FWS program a payment period based prog...,Thank you for your question. Based on the cont...,None,The payment period is applicable to all Title ...,1,1,1,3.373251,c84d7464-b8df-493d-8810-816ce03a02b2,14ea50b3-63e6-4a33-86ae-a690352740ad
9,What is the significance of Volume 8 in the co...,Thank you for your thoughtful question. Based ...,None,Inclusion of clinical work in a standard term ...,1,1,1,3.997215,cf278c0c-f131-4c98-9d5e-27729ab87d7c,7f8551e6-392d-460f-a4a0-dce0a538f442


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.